In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import geopandas as gpd
import geopy
import shapely
from shapely import wkt
from tqdm import tqdm
from shapely.geometry import LineString
from shapely.ops import cascaded_union
from math import radians, cos, sin, asin, sqrt

In [4]:
data_filename = '2022_08_30_РАБОТА_С_ЖД_v21_с_исключениями_отчетная_сводная.xlsx'

In [5]:
def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None


def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

# Graph initialization

In [6]:
nodes = pd.read_excel(data_filename, sheet_name = 'новые пункты')
# nodes = nodes.dropna(subset = ['Идентификатор вершины (глобальный)'])

edges = pd.read_excel(data_filename, sheet_name = 'новые участки')
edges['source_coords'] = edges['source_coords'].astype('str').apply(wkt_loads)
edges['target_coords'] = edges['target_coords'].astype('str').apply(wkt_loads)
edges = edges[edges['исключить'] != 1]

In [7]:
g = nx.Graph()

# adding nodes

for node in nodes.iloc[:,0]:
    if node not in list(g.nodes):
        g.add_node(node)

# adding edges (for loop to add orthodromies)

for idx, edge in edges.iterrows():
    g.add_edge(edge['Идентификатор вершины начала участка (глобальный)'], 
               edge['Идентификатор вершины конца участка (глобальный)'], 
               length = haversine(edge.source_coords.x, edge.source_coords.y,
                        edge.target_coords.x, edge.target_coords.y))

print(f"Vertices: {g.number_of_nodes()}\nEdges: {g.number_of_edges()}")

Vertices: 2463
Edges: 2893


### Station pairs

In [62]:
y2021 = pd.read_excel("distances_calc.xlsb", sheet_name = '2021')
y2022 = pd.read_excel("distances_calc.xlsb", sheet_name = '2022')

station_ids = pd.read_excel("distances_calc.xlsb", sheet_name = 'ПРИВЯЗКИ')

In [63]:
y2021 = pd.merge(y2021, station_ids[['station_name', 'station_id','region_name']],
        left_on = 'source_name', right_on = 'station_name', how = 'left').drop('station_name', axis=1)
y2021 = pd.merge(y2021, station_ids[['station_name', 'station_id', 'region_name']],
         left_on = 'target_name', right_on = 'station_name', how = 'left').drop('station_name', axis=1)

y2021 = y2021.rename(columns = {'station_id_x': 'id_source',
                                'station_id_y': 'id_target',
                                'region_name_x' : 'region_source',
                                'region_name_y': 'region_target'})
y2021.head()

,source_name,target_name,id_source,region_source,id_target,region_target
0,КУЛУНДА-ЭК-Р,ВЛАДИВОСТ-ЭК,48,Алтайский край,6,Приморский край
1,ЗАБАЙК/КИТАЙ,СУЗЕМКА-ЭКСП,24,Забайкальский край,6690,Брянская область
2,ИВАНГР-НАР-Э,НАУШКИ МНР 2,150,Ленинградская область,27,Республика Бурятия
3,ГРОДЕК/КИТАЙ,СБОРНАЯ-УГ.,2363,Приморский край,6598,Тульская область
4,ПОСИНЬ-ЭКС,КАРТАЛЫ 1-ЭК,155,Псковская область,6072,Челябинская область


In [64]:
y2022 = pd.merge(y2022, station_ids[['station_name', 'station_id', 'region_name']],
        left_on = 'source_name', right_on = 'station_name', how = 'left').drop('station_name', axis=1)
y2022 = pd.merge(y2022, station_ids[['station_name', 'station_id', 'region_name']],
         left_on = 'target_name', right_on = 'station_name', how = 'left').drop('station_name', axis=1)

y2022 = y2022.rename(columns = {'station_id_x': 'id_source',
                                'station_id_y': 'id_target',
                                'region_name_x' : 'region_source',
                                'region_name_y': 'region_target'})
y2022.head()

,source_name,target_name,id_source,region_source,id_target,region_target
0,КУЛУНДА-ЭК-Р,ВЛАДИВОСТ-ЭК,48,Алтайский край,6,Приморский край
1,КУЛУНДА-ЭК-Р,КЛЕЩИХА,48,Алтайский край,6020,Новосибирская область
2,ПРИДАЧА,БЕЛОГОРСК,6523,Воронежская область,17,Амурская область
3,ПРИДАЧА,ЧИТА 1,6523,Воронежская область,25,Забайкальский край
4,С-ПЕТЕРБ-ФИН,ВЛАДИВОСТ-ПР,6855,г. Санкт-Петербург,6,Приморский край


In [55]:
y2021 = y2021.dropna(subset = ['id_source', 'id_target'])
y2022 = y2022.dropna(subset = ['id_source', 'id_target'])

# Distance Calculation

In [56]:
def rail_distance(Graph, corr_df):
    
    """
    corr_df: таблица с набором корреспонденций, для которых реализуется расчет дистанции
    
    """
    
    # resulting cols:
    path_distances_list = []
    
    for idx, row in corr_df.iterrows(): 
        
        path_distance = []
        
        from_node = row.id_source
        to_node = row.id_target
        

        # calculates a path distance:
        dist_path = nx.dijkstra_path_length(Graph, from_node, to_node, weight = 'length')
            
        # calculates _straight_ distance (departure and arrival stations only):
#             dist_straight = haversine(row.geom_source.x, row.geom_source.y,
#                         row.geom_target.x, row.geom_target.y)
                    
            
        path_distances_list.append(dist_path)
            
    return path_distances_list

# Multiprocessing

In [57]:
import multiprocessing as mp
from functools import partial
# import istarmap
from tqdm import tqdm
mp.cpu_count() 

8

In [58]:
# list of tuples

id_pairs2021 = list(zip(y2021.id_source, y2021.id_target))
id_pairs2022 = list(zip(y2022.id_source, y2022.id_target))
id_pairs2021[0:5]

[(48, 6), (24, 6690), (150, 27), (2363, 6598), (155, 6072)]

In [59]:
num_workers = mp.cpu_count()  

pool = mp.Pool(num_workers)

with pool as p:
    dijkstra_length = partial(nx.dijkstra_path_length, g, weight = 'length')
    res2021 = p.starmap(dijkstra_length, id_pairs2021)

In [60]:
pool = mp.Pool(num_workers)

with pool as p:
    dijkstra_length = partial(nx.dijkstra_path_length, g, weight = 'length')
    res2022 = p.starmap(dijkstra_length, id_pairs2022)

In [61]:
y2021['distance'] = res2021
y2022['distance'] = res2022
y2021.to_excel('distances_2021.xlsx')
y2022.to_excel('distances_2022.xlsx')